![Intel Logo](images/logo.png)

# Conversion to Model Optimizer

### Contents
    1. Model freezing
    2. Conversion to OpenVINO IR file
    
![Model Optimizr](RES/L02/02.mo.png)

# 0. Prerequisite

In [ ]:
import os
USER = os.environ["USER"]
OVTRANINIG = "/home/{}/openvino_training".format(USER)

!cp RES/L02/mnist_mobilenetv2.h5 $OVTRANINIG/

# 1. Tensorflow Model Freeze

In [ ]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2


model = tf.keras.models.load_model('{}/{}'.format(OVTRANINIG,'mnist_mobilenetv2.h5'))

# Convert Keras model to ConcreteFunction
_model = tf.function(lambda x: model(x)).get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(_model)
frozen_func.graph.as_graph_def()

# Save frozen graph to disk
saved_file = tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir='',
                  name='{}/{}'.format(OVTRANINIG,'mnist_mobilenetv2.pb'),
                  as_text=False)

# 2. Converting and checking the model

In [ ]:
!pip3 install test-generator==0.1.1

In [ ]:
# Set input shape and run Mo
# Tensorflow -> mo_tf.py
# input_model -> frozen file
# input_shape 

#!source /opt/intel/openvino_2021/bin/setupvars.sh
!/opt/intel/openvino_2021/deployment_tools/model_optimizer/mo_tf.py \
    --input_model=$OVTRANINIG/mnist_mobilenetv2.pb \
    --input_shape [1,96,96,3] \
    --output_dir=$OVTRANINIG/

In [ ]:
!cat $OVTRANINIG/mnist_mobilenetv2.xml

In [ ]:
!snap install netron

In [ ]:
!netron $OVTRANINIG/mnist_mobilenetv2.xml

# 3. Converting a different model 